In [142]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [143]:
# Create the optimization model
model = gb.Model("Question 1: Vaccines")

In [144]:
Billy_Bishop_Toronto_City_Airport_costs = [0.05, 0.05, 0.05, 0.05, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07, 0.07, 0.07, 0.07, 0.07, 0.08, 0.08, 0.08, 0.08, 0.08, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1]
Toronto_Pearson_Airport_costs = [0.08, 0.08, 0.08, 0.08, 0.08, 0.05, 0.05, 0.05, 0.05, 0.05, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1, 0.1, 0.07, 0.07, 0.07, 0.07, 0.07, 0.06, 0.06, 0.06, 0.06]

In [145]:
# Create the a single class of decision variables where
# From = {Billy} and To = {29 sites}.
x = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Billy Shipping")
# From = {Pearson} and To = {29 sites}.
y = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Pearson Shipping")

In [146]:
Billy_objective = gb.quicksum(Billy_Bishop_Toronto_City_Airport_costs[j] * x[j] for j in range(29))
Pearson_objective = gb.quicksum(Toronto_Pearson_Airport_costs[j] * y[j] for j in range(29))
model.setObjective(Billy_objective + Pearson_objective, GRB.MINIMIZE)

In [147]:
model.addConstr(gb.quicksum(x[j] for j in range(5)) - gb.quicksum(y[j] for j in range(5)) <= 4800, name="4800 constraint")
model.addConstr(gb.quicksum(y[j] for j in range(5)) - gb.quicksum(x[j] for j in range(5)) <= 4800, name="4800 constraint negative")

<gurobi.Constr *Awaiting Model Update*>

In [148]:
model.addConstr(gb.quicksum(y[j] for j in range(20,25)) <= 8 * (gb.quicksum(x[j] for j in range(10,15))), name="Second constraint")

<gurobi.Constr *Awaiting Model Update*>

In [149]:
model.addConstr(gb.quicksum(x[j] for j in range(25,29)) >= 0.8 * (gb.quicksum(y[j] for j in range(15,20))), name="Third constraint")

<gurobi.Constr *Awaiting Model Update*>

In [150]:
model.addConstr(gb.quicksum(x[j] for j in range(29)) == 100000, "Billy Supply Constraint")
model.addConstr(gb.quicksum(y[j] for j in range(29)) == 250000, "Pearson Supply Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [151]:
#Hosipital Constraint
model.addConstr((gb.quicksum(x[j] + y[j] for j in range(7))) == 4 * (gb.quicksum(x[j] + y[j] for j in range(7,29))), "Hosipital Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [152]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 58 columns and 155 nonzeros
Model fingerprint: 0x5f469efc
Coefficient statistics:
  Matrix range     [8e-01, 8e+00]
  Objective range  [5e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 3e+05]
Presolve removed 2 rows and 49 columns
Presolve time: 0.01s
Presolved: 5 rows, 10 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+03   4.940000e+04   0.000000e+00      0s
       3    1.8452000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.845200000e+04


In [153]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Billy Shipping[4]         4800 
Billy Shipping[5]        25200 
Billy Shipping[9]        70000 
Pearson Shipping[5]       250000 
None


In [154]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  18452.0
